# Benchmarking different quantum compilers


## Load benchmarks

In [35]:
import os
import subprocess
from pathlib import Path

benchmark_files = list(Path(".").rglob("*.qasm"))

## Prepare compilers

In [36]:
compilers = dict()

## QisKit

In [37]:
from qiskit.compiler import transpile

In [38]:
def optimize_qiskit(qasm):
    qc = QuantumCircuit.from_qasm_str(qasm)
    opt_qc = transpile(qc, optimization_level=3, approximation_degree=1.0,
                       basis_gates=['rz', 'rx', 'h', 't', 'tdag', 's', 'cx'])
    return opt_qc.qasm()


compilers['qiskit'] = optimize_qiskit

## PyZX

In [39]:
import pyzx as zx

In [40]:
def optimize_pyzx(qasm):
    c = zx.Circuit.from_qasm(qasm)
    c = c.to_graph()
    zx.simplify.full_reduce(c)
    c.normalize()
    c_opt = zx.extract_circuit(c.copy(), quiet=True, optimize_cnots=3).to_basic_gates()
    return c_opt.to_qasm()


compilers['pyzx'] = optimize_pyzx

## VOQC

To install voqc run
```shell
git submodule update --remote
cd pyvoqc
opam pin voqc https://github.com/inQWIRE/mlvoqc.git#mapping
./install.sh
```
Then copy pyvoqc/lib to your site packages pyvoqc folder

In [41]:
from qiskit import QuantumCircuit
from pyvoqc.qiskit.voqc_pass import voqc_pass_manager

In [42]:
def optimize_voqc(qasm):
    qc = QuantumCircuit.from_qasm_str(qasm)
    vpm = voqc_pass_manager(post_opts=["optimize"])
    qc_opt = vpm.run(qc)
    return qc_opt.qasm()


compilers['voqc'] = optimize_voqc

## $\mid$tket$\rangle$

In [43]:
from pytket.qasm import circuit_from_qasm_str, circuit_to_qasm_str
from pytket.passes import SequencePass, RemoveRedundancies, FullPeepholeOptimise, auto_rebase_pass
from pytket.predicates import CompilationUnit
from pytket import OpType

In [44]:
def optimize_tket(qasm):
    qc = circuit_from_qasm_str(qasm)
    gates = {OpType.Rz, OpType.Rx, OpType.H, OpType.T, OpType.Tdg, OpType.S, OpType.CX}
    rebase = auto_rebase_pass(gates)
    seqpass = SequencePass(
        [
            FullPeepholeOptimise(),
            RemoveRedundancies(),
            rebase,
        ])
    cu = CompilationUnit(qc)
    seqpass.apply(cu)
    qc_opt = cu.circuit
    return circuit_to_qasm_str(qc_opt)


compilers['tket'] = optimize_tket

## Staq

In [45]:
import pystaq


In [46]:
def optimize_staq(qasm):
    qc = pystaq.parse_str(qasm)
    pystaq.simplify(qc)
    pystaq.rotation_fold(qc)
    pystaq.simplify(qc)
    pystaq.cnot_resynth(qc)
    pystaq.simplify(qc)
    return str(qc)


compilers['staq'] = optimize_staq


# Run Benchmarks

## Gate counting

We count gates by finding them in the circuit representation.
Though, we assume a error corrected gate set, meaning we will synthesize any rotations that are not of form $c \times \frac{pi}{4}$ using Peter Sellinger's gridsynth (https://www.mathstat.dal.ca/~selinger/newsynth/)

In [47]:
import subprocess


def gridsynth(theta):
    return subprocess.getoutput(f"./gridsynth {theta} -e 10e-6")

In [48]:
from math import pi

def count_gates(qasm, gate_names):
    qc = QuantumCircuit.from_qasm_str(qasm)
    gates = list(qc)
    count = 0
    for gate in gates:
        gate = gate[0]
        if gate.name in gate_names:
            count += 1
    return count


def approx(a, b):
    return abs(a - b) < 10e-4


def count_gates_synth(theta):
    synth = gridsynth(theta)
    return len(synth) - synth.count('W')


def count_t_synth(theta):
    synth = gridsynth(theta)
    return synth.count('T')


def count_rot_z_t(qasm):
    qc = QuantumCircuit.from_qasm_str(qasm)
    gates = list(qc)
    count = 0
    for gate in gates:
        gate = gate[0]
        if gate.name in ['rz', 'rx', 'u1']:
            theta = gate.params[0]
            if abs(theta % (pi / 4)) < 10e-5:
                count += approx(pi / 4, abs(theta % (pi / 2)))
            else:
                count += count_t_synth(theta)

    return count


def count_t_gates(qasm):
    return count_gates(qasm, ['t', 'tdag']) + count_rot_z_t(qasm)


def count_cnot_gates(qasm):
    return count_gates(qasm, ['cx', 'cz'])


def count_total_gates(qasm):
    qc = QuantumCircuit.from_qasm_str(qasm)
    gates = list(qc)
    count = 0
    for gate in gates:
        gate = gate[0]
        if gate.name in ['rz', 'rx', 'u1']:
            theta = gate.params[0]
            if abs(theta % (pi / 4)) < 10e-5:
                count += 1
            else:
                count += count_gates_synth(theta)
        else:
            count += 1
    return count


In [49]:
f = open("tmp.qasm.x", 'r')
qc2 = f.read()
f.close()

In [50]:
from qiskit.circuit.library import MCMT

#ccz = QuantumCircuit(3)
#ccz = MCMT('cz',2,1)
#ccz = QuantumCircuit.from_qasm_str(optimize_qiskit(ccz.qasm()))
#gate = ccz.to_gate()
#ccz = QuantumCircuit(3)
#ccz.append(gate,[0,1,2])
#print(ccz.qasm())


# Test

qc = QuantumCircuit(2)
qc.rz(0.132312, 0)
qc.t(0)
qc.h(1)
qc.cnot(0, 1)
qc.t(0)
qc.cnot(0, 1)
qc.h(0)
qc.t(0)
qc.t(1)
qc.s(1)
qc.cnot(1, 0)
qc.rx(pi / 4, 1)


print("Original")
print(qc.draw())

qasm = qc.qasm()


for compiler_name in compilers:
    print(compiler_name)
    qasm_opt = compilers[compiler_name](qasm)
    qc_opt = QuantumCircuit.from_qasm_str(qasm_opt)
    print(qc_opt.draw())
    print(count_t_gates(qasm_opt))


Original
     ┌─────────────┐┌───┐     ┌───┐     ┌───┐┌───┐┌───┐           
q_0: ┤ Rz(0.13231) ├┤ T ├──■──┤ T ├──■──┤ H ├┤ T ├┤ X ├───────────
     └────┬───┬────┘└───┘┌─┴─┐└───┘┌─┴─┐├───┤├───┤└─┬─┘┌─────────┐
q_1: ─────┤ H ├──────────┤ X ├─────┤ X ├┤ T ├┤ S ├──■──┤ Rx(π/4) ├
          └───┘          └───┘     └───┘└───┘└───┘     └─────────┘
qiskit
     ┌─────────────┐┌──────────┐┌──────────┐       ┌─────────┐   ┌──────────┐ »
q_0: ┤ Rz(-1.4385) ├┤ Rx(3π/4) ├┤ Rz(-π/2) ├──■────┤ Rx(π/2) ├───┤ Rz(-π/2) ├─»
     └┬────────────┤└──────────┘└──────────┘┌─┴─┐┌─┴─────────┴─┐┌┴──────────┴┐»
q_1: ─┤ Rx(2.2176) ├────────────────────────┤ X ├┤ Rz(0.79021) ├┤ Rx(1.6686) ├»
      └────────────┘                        └───┘└─────────────┘└────────────┘»
«                    
«q_0: ───────────────
«     ┌─────────────┐
«q_1: ┤ Rz(-1.4725) ├
«     └─────────────┘
147
pyzx
     ┌────────────┐┌───┐┌─────────┐┌───┐      ┌───┐                         
q_0: ┤ Rz(1.7031) ├┤ H ├┤ Rz(π/4) ├┤ H ├─■────┤ H ├──

In [56]:
import time

total_time = dict()
total_gates = dict()
total_cnot_gates = dict()
total_t_gates = dict()

base_key = 'base'

# Initalize
total_time[base_key] = 0
total_gates[base_key] = 0
total_cnot_gates[base_key] = 0
total_t_gates[base_key] = 0
for compiler_name_1 in compilers:
    for compiler_name_2 in compilers:
        total_time[compiler_name_1 + compiler_name_2] = 0
        total_gates[compiler_name_1 + compiler_name_2] = 0
        total_cnot_gates[compiler_name_1 + compiler_name_2] = 0
        total_t_gates[compiler_name_1 + compiler_name_2] = 0

# Benchmark individually
completed_benchmarks = []
x = 0 # Limit execution for testing (TODO: REMOVE)
for benchmark_file in benchmark_files:
    if benchmark_file == "temp_in.qasm" or benchmark_file == "tmp.qasm":  # Ignore compiler artifacts
        continue
    print(benchmark_file)
    benchmark = QuantumCircuit.from_qasm_file(benchmark_file).decompose().qasm() # Remove custom gate declarations since tket and voqc don't play nicely with them
    assert not benchmark == ""
    total_gates[base_key] += count_total_gates(benchmark)
    total_cnot_gates[base_key] += count_cnot_gates(benchmark)
    total_t_gates[base_key] += count_t_gates(benchmark)
    for compiler_name_1 in compilers:
        for compiler_name_2 in compilers:
            print(f"{compiler_name_1} + {compiler_name_2}")
            start = time.perf_counter()
            opt_qasm = compilers[compiler_name_1](benchmark)
            if compiler_name_1 != compiler_name_2:
                opt_qasm = compilers[compiler_name_2](benchmark)
            stop = time.perf_counter()
            print(f"Completed in {stop-start} sec")
            total_time[compiler_name_1 + compiler_name_2] += (stop - start)
            total_gates[compiler_name_1 + compiler_name_2] += count_total_gates(opt_qasm)
            total_cnot_gates[compiler_name_1 + compiler_name_2] += count_cnot_gates(opt_qasm)
            total_t_gates[compiler_name_1 + compiler_name_2] += count_t_gates(opt_qasm)
    completed_benchmarks += str(benchmark_file)
    x += 1
    if x >= 3:
        break

# Print results
print(total_time)
print(total_gates)
print(total_cnot_gates)
print(total_t_gates)

Nam-benchmarks/Arithmetic_and_Toffoli/adder_8.qasm
qiskit + qiskit
Completed in 1.087130759000047 sec
qiskit + pyzx
Completed in 7.338517799000101 sec
qiskit + voqc
Completed in 1.700059334999878 sec
qiskit + tket
Completed in 3.206322266999905 sec
qiskit + staq
Completed in 1.3225662330000887 sec
pyzx + qiskit
Completed in 7.161215027000026 sec
pyzx + pyzx
Completed in 6.395087568000008 sec
pyzx + voqc
Completed in 6.6133201170000575 sec
pyzx + tket
Completed in 8.709096212000077 sec
pyzx + staq
Completed in 8.206761061999941 sec
voqc + qiskit
Completed in 2.2705168249999588 sec
voqc + pyzx
Completed in 9.484389915999827 sec
voqc + voqc
Completed in 0.8090161009999974 sec
voqc + tket
Completed in 3.3905775849998463 sec
voqc + staq
Completed in 0.8207794480001667 sec
tket + qiskit
Completed in 3.8866139990000192 sec
tket + pyzx
Completed in 10.396440009999878 sec
tket + voqc
Completed in 3.0290468050000072 sec
tket + tket
Completed in 2.2857636300000195 sec
tket + staq
Completed in 2.2

In [59]:
import pickle

def get_new_file(base_name):
    i = 0
    while os.path.exists(f"{base_name}{i}.pkl"):
        i += 1
    return open(f"{base_name}{i}.pkl", 'wb')

pickle.dump(completed_benchmarks, get_new_file("completed_benchmarks"))
pickle.dump(total_time, get_new_file("total_time"))
pickle.dump(total_gates, get_new_file("total_gates"))
pickle.dump(total_t_gates, get_new_file("total_t_gates"))
pickle.dump(total_cnot_gates, get_new_file("total_cnot_gates"))